# The basics of using IQM Resonance

You will have free access to IQM's cloud quantum computing platform, IQM Resonance, during this hackathon. 

In this notebook, you will see how to connect to IQM Resonance and how to manage your jobs in different frameworks. For each framework, where these features are supported, you will learn to:
- Connect to Resonance
- Submit a job
- Submit several circuits in the same job
- Retrieve jobs you have already finished

The frameworks currently listed in this tutorial are:
- Qrisp
- CUDA-Q
- Qiskit
- Cirq

You are welcome to use any of them for this challenge, or any other framework that supports IQM Resonance as a backend.

If you run into any problems (there are lots of things that can go wrong here!), please get in touch with an IQM mentor.

# Setting up your Python environment to use IQM Resonance

You will need to set up a **Python 3.11 or 3.12** runtime environment, neither of which is the most recent Python version, with the appropriate packages installed. You will need to either:
- Install one of these Python versions from the Python website
- Set up a conda or other virtual environment in one of these versions
- Run your code via qBraid (see the qBraid tutorial from Friday)
- Run your code via Google Colab, in which case you should change the runtime type to 2025.07.

If you are using CUDA-Q, we strongly recommend using qBraid, although Google Colab will also work fine.

# Making an IQM Resonance account and an API token

In order to connect to IQM Resonance, you will need an account and an API token. Your API token is a string, unique to you, that associates any jobs you submit to you account. Your API token is required to submit a job to Resonance.

### Create your account

Use [this link](https://resonance.meetiqm.com/sign-up/iquhack26-iqm%20challenge) to create your account. This link should give you plenty of credits to complete the challenge. (If you run out of credits for some reason, let an IQM mentor know.)

### Creating your API token

To create your API token,

1) Go to [the Resonance website](resonance.meetiqm.com) and log in.
2) Click your initials in the upper right corner.
3) In the menu that pops up, under "API token", click the circular arrows.
4) You will now see your API token. Copy it somewhere safe!

### Using your API token

Your API token will work for every job you submit until the expiration date listed on the Resonance website. In the code snippets below where we submit jobs to Resonance, just copy-paste your API token when prompted. 

### I lost or gave away my API token! Now what?

If you ever lose your API token (or if someone else gets your API token), you can make a new one by repeating the steps in "Creating your API token." This will cause your old API token to stop working.

# Submitting jobs to Resonance

Scroll to your preferred framework to see how to submit jobs to Resonance.

## Qrisp

Install required packages:

In [1]:
%pip install --upgrade pip
%pip install iqm-client
%pip install "qrisp[iqm]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Build a simple quantum circuit:

In [2]:
from qrisp import *

# ---- From your subgraph selector output ----
NODES = [
    "QB12","QB14","QB18","QB19","QB20",
    "QB21","QB22","QB26","QB27","QB35"
]
ROOT = "QB26"

CZ_ORDER_BFS = [
    ("QB26","QB18"),
    ("QB18","QB19"),
    ("QB19","QB20"),
    ("QB19","QB27"),
    ("QB20","QB12"),
    ("QB20","QB21"),
    ("QB27","QB35"),
    ("QB21","QB22"),
    ("QB22","QB14")
]

def build_tree_ghz():
    """Build a GHZ state on the spanning tree defined by CZ_ORDER_BFS."""
    qv_local = QuantumVariable(len(NODES))
    idx = {name: i for i, name in enumerate(NODES)}

    # 1) Put root into superposition
    h(qv_local[idx[ROOT]])

    # 2) Spread entanglement along the BFS tree order
    # Use cx here (Qrisp will compile it to native CZ on IQM backends)
    for u, v in CZ_ORDER_BFS:
        cx(qv_local[idx[u]], qv_local[idx[v]])

    return qv_local

# Build the circuit once and show it
qv = build_tree_ghz()
print(qv.qs)


QuantumCircuit:
---------------
                                     ┌───┐               
qv_local.0: ─────────────────────────┤ X ├───────────────
                                     └─┬─┘          ┌───┐
qv_local.1: ───────────────────────────┼────────────┤ X ├
                 ┌───┐                 │            └─┬─┘
qv_local.2: ─────┤ X ├──■──────────────┼──────────────┼──
                 └─┬─┘┌─┴─┐            │              │  
qv_local.3: ───────┼──┤ X ├──■────■────┼──────────────┼──
                   │  └───┘┌─┴─┐  │    │              │  
qv_local.4: ───────┼───────┤ X ├──┼────■────■─────────┼──
                   │       └───┘  │       ┌─┴─┐       │  
qv_local.5: ───────┼──────────────┼───────┤ X ├──■────┼──
                   │              │       └───┘┌─┴─┐  │  
qv_local.6: ───────┼──────────────┼────────────┤ X ├──■──
            ┌───┐  │              │            └───┘     
qv_local.7: ┤ H ├──■──────────────┼──────────────────────
            └───┘               ┌─┴─┐   

Quick sanity tests on the simulator (no Resonance backend):


In [3]:
# ---- Test A (Z basis): GHZ should be mostly 0...0 and 1...1 ----
res_z_sim = qv.get_measurement()
all0 = "0" * len(NODES)
all1 = "1" * len(NODES)
p_all0 = res_z_sim.get(all0, 0.0)
p_all1 = res_z_sim.get(all1, 0.0)

print("=== Simulator Test A: Z-basis GHZ check ===")
print("P(all-0) =", p_all0)
print("P(all-1) =", p_all1)
print("P(all-0)+P(all-1) =", p_all0 + p_all1)
print("Top 5 outcomes:", sorted(res_z_sim.items(), key=lambda x: -x[1])[:5])
print()

# ---- Test B (X basis parity): rotate all qubits to X basis, then measure ----
qv_x_sim = build_tree_ghz()
for i in range(len(NODES)):
    h(qv_x_sim[i])

res_x_sim = qv_x_sim.get_measurement()
parity_even_sim = sum(p for bitstr, p in res_x_sim.items() if bitstr.count("1") % 2 == 0)

print("=== Simulator Test B: X-basis parity check ===")
print("Even parity probability =", parity_even_sim)
print("Top 5 outcomes:", sorted(res_x_sim.items(), key=lambda x: -x[1])[:5])


=== Simulator Test A: Z-basis GHZ check ===                                          
P(all-0) = 0.5
P(all-1) = 0.5
P(all-0)+P(all-1) = 1.0
Top 5 outcomes: [('0000000000', 0.5), ('1111111111', 0.5)]

=== Simulator Test B: X-basis parity check ===                                       
Even parity probability = 1.0000000000000002
Top 5 outcomes: [('0000000000', 0.0019531250000000004), ('1100000000', 0.0019531250000000004), ('1010000000', 0.0019531250000000004), ('0110000000', 0.0019531250000000004), ('1001000000', 0.0019531250000000004)]


Connect to Resonance and submit the job:

In [4]:
from qrisp.interface import IQMBackend

quantum_computer = IQMBackend(
    api_token=input("IQM Resonance API token"),
    device_instance="emerald"   # Change this to select the device you run on
)

# Run the GHZ circuit in Z basis on hardware
res_z_hw = qv.get_measurement(backend=quantum_computer)

all0 = "0" * len(NODES)
all1 = "1" * len(NODES)
p_all0 = res_z_hw.get(all0, 0.0)
p_all1 = res_z_hw.get(all1, 0.0)

print("=== Hardware Test A: Z-basis GHZ check ===")
print(res_z_hw)
print("P(all-0) =", p_all0)
print("P(all-1) =", p_all1)
print("P(all-0)+P(all-1) =", p_all0 + p_all1)
print("Top 5 outcomes:", sorted(res_z_hw.items(), key=lambda x: -x[1])[:5])


/Users/tristancarter/2026-IQM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Progress in queue:   0%|          | 0/1 [00:04<?, ?it/s]


=== Hardware Test A: Z-basis GHZ check ===
{'0000000000': 0.393, '1111111111': 0.241, '1111111101': 0.057, '0000000011': 0.021, '1111111011': 0.02, '1111111100': 0.018, '0000000100': 0.015, '0100000000': 0.014, '0111111111': 0.014, '0011000111': 0.013, '0010000100': 0.011, '1100111000': 0.01, '0000000010': 0.009, '1111111110': 0.008, '1101111111': 0.008, '1000000000': 0.007, '0010000000': 0.007, '0100111000': 0.007, '1101111011': 0.007, '1011100111': 0.007, '1011111111': 0.007, '1111101111': 0.006, '0100001000': 0.005, '0100011000': 0.005, '1110111100': 0.005, '1011110111': 0.005, '1110111111': 0.005, '1111011111': 0.004, '0000001000': 0.003, '1111111001': 0.003, '0001000011': 0.003, '0000010000': 0.002, '1111111000': 0.002, '0011000100': 0.002, '0110011100': 0.002, '0110111100': 0.002, '0000000001': 0.002, '1011100101': 0.002, '1111101101': 0.002, '0110111111': 0.002, '0110000000': 0.001, '0001000000': 0.001, '1001000000': 0.001, '0000100000': 0.001, '1000100000': 0.001, '1100110000':

Submit several circuits in a single job:

In [5]:
# Submit several circuits in a single job:
#   Circuit 1: GHZ measured in Z basis
#   Circuit 2: GHZ measured in X basis (apply H on all qubits before measurement)

circuit_list = []

# Circuit 1 (Z)
qv_z = build_tree_ghz()
circuit_list.append(qv_z)

# Circuit 2 (X): rotate to X basis
qv_x = build_tree_ghz()
for i in range(len(NODES)):
    h(qv_x[i])
circuit_list.append(qv_x)

# Show the X-basis circuit as an example
print(circuit_list[1].qs)

jobs = batched_measurement(circuit_list, backend=quantum_computer)


QuantumCircuit:
---------------
                                     ┌───┐┌───┐               
qv_local.0: ─────────────────────────┤ X ├┤ H ├───────────────
                                     └─┬─┘└───┘     ┌───┐┌───┐
qv_local.1: ───────────────────────────┼────────────┤ X ├┤ H ├
                 ┌───┐     ┌───┐       │            └─┬─┘└───┘
qv_local.2: ─────┤ X ├──■──┤ H ├───────┼──────────────┼───────
                 └─┬─┘┌─┴─┐└───┘       │  ┌───┐       │       
qv_local.3: ───────┼──┤ X ├──■────■────┼──┤ H ├───────┼───────
                   │  └───┘┌─┴─┐  │    │  └───┘┌───┐  │       
qv_local.4: ───────┼───────┤ X ├──┼────■────■──┤ H ├──┼───────
                   │       └───┘  │       ┌─┴─┐└───┘  │  ┌───┐
qv_local.5: ───────┼──────────────┼───────┤ X ├──■────┼──┤ H ├
                   │              │       └───┘┌─┴─┐  │  ├───┤
qv_local.6: ───────┼──────────────┼────────────┤ X ├──■──┤ H ├
            ┌───┐  │  ┌───┐       │            └───┘     └───┘
qv_local.7: ┤ H ├──■──┤

Progress in queue:   0%|          | 0/1 [00:04<?, ?it/s]


Retrieve the measurement results:

In [6]:
# Retrieve the measurement results from the batched job

z_res = jobs[0]  # Z-basis GHZ measurement results
x_res = jobs[1]  # X-basis GHZ measurement results

print("=== Batched results (Z basis) ===")
print(z_res)
all0 = "0" * len(NODES)
all1 = "1" * len(NODES)
print("P(all-0)+P(all-1) =", z_res.get(all0, 0.0) + z_res.get(all1, 0.0))
print("Top 5 outcomes:", sorted(z_res.items(), key=lambda x: -x[1])[:5])
print()

print("=== Batched results (X basis) ===")
print(x_res)
parity_even = sum(p for bitstr, p in x_res.items() if bitstr.count("1") % 2 == 0)
print("Even parity probability =", parity_even)
print("Top 5 outcomes:", sorted(x_res.items(), key=lambda x: -x[1])[:5])


=== Batched results (Z basis) ===
{'0000000000': 0.395, '1111111111': 0.247, '1111111101': 0.04, '1111111011': 0.028, '0111111111': 0.021, '0011000111': 0.013, '1110111111': 0.013, '0010000100': 0.012, '1101111011': 0.012, '1011100111': 0.011, '1000000000': 0.009, '1100111000': 0.009, '0000000001': 0.009, '0000000011': 0.009, '0100000000': 0.008, '0000000010': 0.008, '0000000100': 0.007, '1111111100': 0.007, '0010000000': 0.006, '0000010000': 0.006, '1111111110': 0.006, '1011111111': 0.006, '0100011000': 0.005, '1011000111': 0.005, '1101111111': 0.005, '0001000011': 0.004, '1011110111': 0.004, '1111110111': 0.004, '1111101111': 0.004, '0000001000': 0.003, '0100001000': 0.003, '1110111100': 0.003, '1110111101': 0.003, '0010000111': 0.003, '0001000000': 0.002, '1101111000': 0.002, '0110011100': 0.002, '1101111001': 0.002, '1111111001': 0.002, '1101111101': 0.002, '0111111101': 0.002, '1110111011': 0.002, '0101111011': 0.002, '0111111011': 0.002, '1111011111': 0.002, '0110111111': 0.002, 

As of December 2025, retrieving a job via job ID is not supported in qrisp.

# CUDA-Q

Note that CUDA-Q does not support Star topology devices, in particular Sirius is not supported.

Install required packages:

In [ ]:
%pip install cudaq
%pip install iqm-cortex-cli

Build a simple quantum circuit:

In [ ]:
import cudaq

@cudaq.kernel
def simpleCircuit():
    qubits = cudaq.qvector(3)
    h(qubits[0])
    x.ctrl(qubits[0],qubits[1])
    x.ctrl(qubits[0],qubits[2])
    mz(qubits)

Connect to IQM Resonance and submit the job:

In [ ]:
import os

os.environ["IQM_TOKEN"] = input("Input your Resonance API token: ")

iqm_url = "https://cocos.resonance.meetiqm.com/emerald" # You can change Emerald to other machines if you want!

cudaq.set_target("iqm", url=iqm_url)

result = cudaq.sample(simpleCircuit)

Submit several circuits to retrieve later (without restarting your runtime):

In [ ]:
import numpy as np

num_circuits=5
phases = [2*np.pi*x/num_circuits for x in range(num_circuits) ]

job_list = []
@cudaq.kernel
def phaseCircuit(phase:float):
    qubit = cudaq.qubit()
    # Create a quantum circuit that applies a Hadamard, a phase gate, and another Hadamard to a single qubit    qc.h(0)
    h(qubit)
    rz(phase,qubit)
    h(qubit)
    mz(qubit)

for phase in phases:
    job_list.append(cudaq.sample_async(phaseCircuit, phase))

Retrieve the jobs:

In [ ]:
for job in job_list:
    print(job.get())

As of January 2026, neither batched jobs nor retrieving a job via job ID are supported in CUDA-Q.

# Qiskit

Install required packages:

In [ ]:
!pip install "iqm-client[qiskit]"

Build a simple quantum circuit:

In [ ]:
from qiskit import *

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
qc.measure_all()

Connect to IQM Resonance and submit the job:

In [ ]:
from iqm.qiskit_iqm import IQMProvider
provider = IQMProvider("https://resonance.meetiqm.com",
                       quantum_computer="emerald",
                       token=input("Input your IQM Resonance token"))
backend = provider.get_backend()

qc_transpiled = transpile(qc,backend)
job = backend.run(qc_transpiled)

results=job.result()
print(results.get_counts())

Submit several circuits in a single job:

In [ ]:
import numpy as np
circuits = []

num_circuits=5
phases = [2*np.pi*x/num_circuits for x in range(num_circuits) ]

circuit_list = []
for phase in phases:
    qc = QuantumCircuit(1)
    # Create a quantum circuit that applies a Hadamard, a phase gate, and another Hadamard to a single qubit    qc.h(0)
    qc.h(0)
    qc.p(phase,0)
    qc.h(0)
    qc.measure_all()
    qc_transpiled = qc.transpile(backend=backend) # Get the circuit ready for IQM hardware
    circuit_list.append(qc_transpiled)

#show a circuit as an example
circuit_list[2].draw("mpl")
    
jobs = backend.run(circuit_list)

Retrieve the jobs:

NOTE: If you restarted your runtime after submitting the job, you need the Resonance job ID. It can be obtained as shown below, or, if you click on the job in your list of Resonance jobs, it is the long hexadecimal string at the top of the pop-up window. (Note: in the list of jobs, you will see a 6-character job ID. This is not the full job ID! You need the 32-character (plus hyphens) job ID.)

In [ ]:
# When working in the same runtime:
print(jobs.result().get_counts())

# Or you can use the job ID, even if you restarted the runtime:
job_id = jobs.job_id()  # For this notebook we're getting the job ID this way,
                        # but you can get it from your list of past jobs in Resonance
print(job_id)

retrieved_job = backend.retrieve_job(job_id)

print(retrieved_job.result().get_counts())

# Cirq

Install required packages:

In [ ]:
!pip install iqm-client[cirq]

Connect to IQMResonance (Note: there are several ways to build the circuit in Cirq. One way is to use arbitrary qubits that don't depend on the backend, but it's a little simpler to use backend-specific qubits instead. In order to do this, we cannot build the circuit before connecting to the backend.)

In [ ]:
from iqm.cirq_iqm.iqm_sampler import IQMSampler

server_url = "https://resonance.meetiqm.com/"
sampler=IQMSampler(server_url,
                   quantum_computer="emerald",
                   token=input("Input your IQM Resonance token"))

Build a simple circuit

In [ ]:
import cirq

qubits = sampler.device.qubits[:3]

circuit = cirq.Circuit()

circuit.append(cirq.H(qubits[0]))

for qb in qubits[1:]:
    circuit.append(cirq.CNOT(qubits[0],qb))

circuit.append(cirq.measure(*qubits, key='result'))

Run the circuit:

In [ ]:
routed_circuit, _, _ = sampler.device.route_circuit(circuit)

decomposed_circuit = sampler.device.decompose_circuit(routed_circuit)

from iqm.cirq_iqm.optimizers import simplify_circuit
simplified_circuit = simplify_circuit(decomposed_circuit)

print(simplified_circuit) 

shots=1000
result = sampler.run(simplified_circuit, repetitions=shots)

print(result.histogram(key='result'))

Submit several circuits in a single job:

In [ ]:
import numpy as np
circuits = []

num_circuits=5
phases = [2*np.pi*x/num_circuits for x in range(num_circuits) ]

circuit_list = []
for phase in phases:
    qubit = sampler.device.qubits[:1]
    # Create a quantum circuit that applies a Hadamard, a phase gate, and another Hadamard to a single qubit
    c = cirq.Circuit(
        cirq.H(qubit[0]),
        cirq.rz(phase)(qubit[0]),
        cirq.H(qubit[0]),
        cirq.measure(*qubit,key="result")
    )
    routed_c, _, _ = sampler.device.route_circuit(c)
    decomposed_c = sampler.device.decompose_circuit(routed_c)
    simplified_c = simplify_circuit(decomposed_c)
    circuit_list.append(simplified_c)

#show a circuit as an example
print(circuit_list[2])

shots=1000
result = sampler.run_iqm_batch(circuit_list, repetitions=shots)

for res in result:
    print(res.histogram(key="result") )

As of December 2025, retrieving jobs via their job ID is not supported in Cirq.